# preprocessing

In [ ]:
from kmer_feature_matrix_pipeline_funcs import *
from filtering_functions import *
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('/Users/joesouber/OneDrive - University of Bristol/MSc Data Science/data science mini project/dsmp-2024-group-13/vdjdb_full.txt', sep='\t')
df = pd.DataFrame(df)

df_preprocessed = preprocess_data(df)
df_preprocessed_species = filter_by_species(df_preprocessed)
df_preprocessed_species_min_score = filter_by_minimum_score(df_preprocessed_species)
df_filtered_epitope = filter_by_length_range(df_preprocessed_species_min_score, 'antigen.epitope')
df_done = filter_by_mhc_class(df_preprocessed_species_min_score)
# Assuming 'df' is your original DataFrame
min_instances = 5 # Set the minimum number of instances required for inclusion
filtered_df = filter_by_epitope_instances(df_done, label_col='antigen.epitope', min_instances=min_instances)

print(f"Original DataFrame had {len(df_done)} rows.")
print(f"Filtered DataFrame has {len(filtered_df)} rows.")
print(f"unique epitopes in the filtered DataFrame: {filtered_df['antigen.epitope'].value_counts()}")
print(f"Number of unique epitopes in the filtered DataFrame: {filtered_df['antigen.epitope'].nunique()}")

# ALPHA

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

X, y, feature_names, kmer_count_dict, epitope_names, epitope_to_int = create_features_matrix(filtered_df, include_alpha=False, include_beta=True, alpha_col='cdr3.alpha', beta_col='cdr3.beta', label_col='antigen.epitope', k=3)
classifier = GradientBoostingClassifier(subsample=0.8,random_state=10, n_estimators=200, learning_rate=0.01, max_features='sqrt', max_depth=7)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
auc_dict1, acc_list, precision_list, recall_list, all_class_reports, all_conf_matrices, clf, misclassified_instances, misclassified_details, X_train, X_test = predict_auc(X, y, classifier, 5, epitope_names, True)

# BETA

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

X, y, feature_names, kmer_count_dict, epitope_names, epitope_to_int = create_features_matrix(filtered_df, include_alpha=True, include_beta=False, alpha_col='cdr3.alpha', beta_col='cdr3.beta', label_col='antigen.epitope', k=3)
classifier = GradientBoostingClassifier(subsample=0.8,random_state=10, n_estimators=200, learning_rate=0.01, max_features='sqrt', max_depth=7)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
auc_dict2, acc_list, precision_list, recall_list, all_class_reports, all_conf_matrices, clf, misclassified_instances, misclassified_details, X_train, X_test = predict_auc(X, y, classifier, 5, epitope_names, True)

# combined


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

X, y, feature_names, kmer_count_dict, epitope_names, epitope_to_int = create_features_matrix(filtered_df, include_alpha=True, include_beta=True, alpha_col='cdr3.alpha', beta_col='cdr3.beta', label_col='antigen.epitope', k=3)
classifier = GradientBoostingClassifier(subsample=0.8,random_state=10, n_estimators=200, learning_rate=0.01, max_features='sqrt', max_depth=7)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
auc_dict3, acc_list, precision_list, recall_list, all_class_reports, all_conf_matrices, clf, misclassified_instances, misclassified_details, X_train, X_test = predict_auc(X, y, classifier, 5, epitope_names, True)


# Boxplots comparison of alpha, beta and alphabeta

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# where auc_dict 1 is alpha, aucdict2 is beta and aucdict3 is alpha+beta
def extract_average_auc(auc_dict):
    """ Extracts and calculates average AUC per fold from a given AUC dictionary. """
    return [np.mean(list(fold.values())) for fold in auc_dict.values()]


# Assuming auc_dict, auc_dict2, auc_dict3 are defined elsewhere and imported here
average_auc_alpha = extract_average_auc(auc_dict1)      # from alpha
average_auc_beta = extract_average_auc(auc_dict2)      # from beta
average_auc_alpha_beta = extract_average_auc(auc_dict3)  # from alpha+beta

# Combine the results into a list of lists for plotting
data = [average_auc_alpha, average_auc_beta, average_auc_alpha_beta]
labels = ['Alpha', 'Beta', 'Alpha+Beta']

# Plotting
plt.figure(figsize=(10, 6))
sns.boxplot(data=data)
plt.xticks(ticks=range(3), labels=labels)
plt.title('Comparison of Average AUC across Different Feature Matrix Methods')
plt.xlabel('Method')
plt.ylabel('Average AUC')
plt.show()
plt.savefig('/content/alpha_vs_beta_vs_alphabeta.png')

# top and bottom performing epitopes for each method (change auc_dict to specify between methods)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Extracting average AUC values
average_auc_per_fold = [np.mean(list(fold.values())) for fold in auc_dict3.values()]

# Calculating epitope-specific AUCs across folds
epitope_aucs = {epi: [] for epi in range(len(epitope_names))}  # Initialize dictionary with epitope indices
for fold in auc_dict3.values():
    for epitope, auc in fold.items():
        if isinstance(epitope, int):  # Filtering out 'micro' and 'macro'
            epitope_aucs[epitope].append(auc)

# Calculating average AUC for each epitope
average_auc_per_epitope = {epi: np.mean(aucs) for epi, aucs in epitope_aucs.items()}
sorted_epitopes = sorted(average_auc_per_epitope.items(), key=lambda item: item[1])

# Extracting top 3 and bottom 3 epitopes, converting index to names
top_3_epitopes = [(epitope_names[epi], auc) for epi, auc in sorted_epitopes[-3:]]
bottom_3_epitopes = [(epitope_names[epi], auc) for epi, auc in sorted_epitopes[:3]]

# Converting to a list for plotting using names
top_3_values = [epitope_aucs[epi] for epi, _ in sorted_epitopes[-3:]]
bottom_3_values = [epitope_aucs[epi] for epi, _ in sorted_epitopes[:3]]

# Plotting
plt.figure(figsize=(14, 6))



plt.subplot(1, 2, 1)
sns.boxplot(data=top_3_values)
plt.title('Top 3 Performing Epitopes')
plt.xticks(ticks=range(3), labels=[name for name, _ in top_3_epitopes], rotation=90)
plt.xlabel('Epitopes')
plt.ylabel('AUC')
plt.savefig('/content/alpha_beta_epitopes_topperformance.png')

plt.subplot(1, 2, 2)
sns.boxplot(data=bottom_3_values)
plt.title('Bottom 3 Performing Epitopes ')
plt.xticks(ticks=range(3), labels=[name for name, _ in bottom_3_epitopes], rotation=90)
plt.xlabel('Epitopes')
plt.ylabel('AUC')
plt.savefig('/content/apha_beta_epitopes_bottomperformance.png')

plt.tight_layout()
plt.show()

# top feature importances

In [ ]:
plot_feature_importance(clf, feature_names, top_n=20)

# positional frequencies of top importance kmers

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

importances = clf.feature_importances_

    # Create a list of tuples (feature_name, importance)
feature_importance = list(zip(feature_names, importances))

    # Sort the feature importances by most important first
feature_importance = sorted(feature_importance, key=lambda x: x[1], reverse=True)

    # Taking the top n features
top_features = feature_importance[:50]
features, scores = zip(*top_features)
kmer_data = pd.DataFrame(list(kmer_count_dict.items()), columns=['kmer_pos', 'count'])
kmer_data['kmer'], kmer_data['position'] = zip(*kmer_data['kmer_pos'].apply(lambda x: (x.split('_')[0], int(x.split('_')[1]))))

# Convert `top_features` to uppercase to match the case in `kmer_data`
top_kmers_upper = [f"{feat.split('_')[0].upper()}_{feat.split('_')[1]}" for feat, importance in top_features]

# Filter `kmer_data` to include only the top k-mers
top_kmer_data = kmer_data[kmer_data['kmer_pos'].isin(top_kmers_upper)]

# Create the pivot table for the heatmap
if not top_kmer_data.empty:
    top_kmer_pivot = top_kmer_data.pivot(index='kmer_pos', columns='position', values='count').fillna(0)

    # Generate the heatmap with the top 50 k-mers
    plt.figure(figsize=(20, 10))
    sns.set(font_scale=1)
    sns.heatmap(top_kmer_pivot, annot=True, cmap='YlGnBu', fmt=".0f", linewidths=.5, cbar_kws={'shrink': 0.5})
    plt.title('Frequency of Top 50 k-mers Across Positions')
    plt.xlabel('Position in Sequence')
    plt.ylabel('k-mer')
    plt.show()
else:
    print("No matching k-mers found in the dataset. Please check the feature names and dataset.")


# zoom in on specific kmer

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



# Define the specific k-mer to visualize, e.g., 'ssi'
specific_kmer = 'LIF'

# Filter for all positions of the specific k-mer
specific_kmer_data = kmer_data[kmer_data['kmer_pos'].str.contains(f'^{specific_kmer}_', case=False, regex=True)]

# Create the pivot table for the heatmap
if not specific_kmer_data.empty:
    # Extract k-mer and position
    specific_kmer_data['position'] = specific_kmer_data['kmer_pos'].apply(lambda x: int(x.split('_')[1]))
    
    # Pivot by position
    kmer_position_pivot = specific_kmer_data.pivot_table(index='kmer_pos', columns='position', values='count', aggfunc='sum').fillna(0)

    # Generate the heatmap
    plt.figure(figsize=(10, 5))
    sns.set(font_scale=1)
    sns.heatmap(kmer_position_pivot, annot=True, cmap='YlGnBu', fmt=".0f", linewidths=.5, cbar_kws={'shrink': 0.5})
    plt.title(f'Frequency of {specific_kmer} Across Positions')
    plt.xlabel('Position in Sequence')
    plt.ylabel('K-mer and Position')
    plt.show()
else:
    print("No matching positions found for the specified k-mer. Please check the feature names and dataset.")


# kmers in multiple positions

In [ ]:
import pandas as pd



# Extract k-mer names from kmer_pos by removing the position suffix
kmer_data['kmer'] = kmer_data['kmer_pos'].apply(lambda x: x.split('_')[0])

# Group by the extracted k-mer names and count unique positions for each k-mer
kmer_position_counts = kmer_data.groupby('kmer')['kmer_pos'].nunique()

# Filter to find k-mers that appear in more than one position
kmers_in_multiple_positions = kmer_position_counts[kmer_position_counts > 1]
kmers_in_multiple_positions_sorted = kmers_in_multiple_positions.sort_values(ascending=False)
# Print the result
kmers_in_multiple_positions_sorted

# kmers in multiple positions that are also in top importance (can specify top importance value, e.g top 20)

In [ ]:
import pandas as pd



# Extract k-mer names from kmer_pos by removing the position suffix
kmer_data['kmer'] = kmer_data['kmer_pos'].apply(lambda x: x.split('_')[0])

# Group by the extracted k-mer names and count unique positions for each k-mer
kmer_position_counts = kmer_data.groupby('kmer')['kmer_pos'].nunique()

# Filter to find k-mers that appear in more than one position
kmers_in_multiple_positions = kmer_position_counts[kmer_position_counts > 1]
kmers_in_multiple_positions_sorted = kmers_in_multiple_positions.sort_values(ascending=False)
# Print the result
kmers_in_multiple_positions_sorted

# kmers in both top importance list and  top multiple positions list

In [ ]:
# Extracting k-mer name from kmer_pos and aggregating total count for each k-mer
kmer_data['kmer'] = kmer_data['kmer_pos'].apply(lambda x: x.split('_')[0])
total_counts = kmer_data.groupby('kmer')['count'].sum().reset_index()

# Convert kmer to uppercase to standardize (assuming most populous already in uppercase)
total_counts['kmer'] = total_counts['kmer'].str.upper()
most_populous_kmers = total_counts.sort_values(by='count', ascending=False).head(30)

# Assuming `clf` is your trained classifier and `feature_names` are available
importances = clf.feature_importances_  # Example feature importances, replace with actual
#feature_names = ['ssi_0', 'agy_1', 'gyg_2', 'ygg_3', 'ggs_4', 'qgn_5', 'ssi_6', 'agy_7', 'gyg_8', 'ygg_9'] * 15

# Create a list of tuples (feature_name, importance) and sort by importance
feature_importance = list(zip(feature_names, importances))
feature_importance_sorted = sorted(feature_importance, key=lambda x: x[1], reverse=True)
top_features = feature_importance_sorted[:30]

# Convert `top_features` to uppercase k-mer names without positions for comparison
top_kmers_upper = [feat[0].split('_')[0].upper() for feat in top_features]

# Find common k-mers
common_kmers = set(most_populous_kmers['kmer']).intersection(top_kmers_upper)
print("Common k-mers in both top populous and top important lists:", common_kmers)

# tying it all together

In [ ]:
pip install matplotlib-venn

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns

# Generate the plots

## First, the Venn diagram
plt.figure(figsize=(8, 8))
venn_labels = {'10': len(most_populous_kmers) - len(common_kmers), 
               '01': len(top_kmers_upper) - len(common_kmers), 
               '11': len(common_kmers)}
venn2(subsets=venn_labels, set_labels=('Most Populous K-mers', 'Top Important K-mers'))
plt.title("Venn Diagram of K-mer Overlap")
plt.show()

## Then, the bar plot for the most populous k-mers
plt.figure(figsize=(12, 8))
sns.barplot(x='kmer', y='count', data=most_populous_kmers.head(30), 
            palette=["red" if kmer in common_kmers else "blue" for kmer in most_populous_kmers['kmer'].head(30)])
plt.xticks(rotation=45)
plt.title('Top 30 Most Populous K-mers (Red indicates Top Importance)')
plt.xlabel('K-mer')
plt.ylabel('Total Count')
plt.show()

# task is now to get out results from SETE and TCRdist, plus possibly langauge approach, so that we can output a combined final graph

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# AUC values for each method
auc_no_pca = np.array([0.772, 0.741, 0.768, 0.807, 0.815])
auc_pca = np.array([0.793, 0.774, 0.741, 0.751, 0.748])
auc_umap = np.array([0.639, 0.581, 0.643, 0.677, 0.65])
auc_sete = np.array([0.767,0.696,0.751,0.707])
auc_tcrdist = np.array([0.761838, 0.749288,	0.770169,	0.762701, 0.76629	])

# Calculate averages
avg_no_pca = np.mean(auc_no_pca)
avg_pca = np.mean(auc_pca)
avg_umap = np.mean(auc_umap)
avg_sete = np.mean(auc_sete)
avg_tcrdist = np.mean(auc_tcrdist)

# Calculate percentage differences
perc_diff_pca_no_pca = ((avg_pca - avg_no_pca) / avg_no_pca) * 100
perc_diff_umap_no_pca = ((avg_umap - avg_no_pca) / avg_no_pca) * 100
perc_diff_sete_no_pca = ((avg_sete - avg_no_pca) / avg_no_pca) * 100
perc_diff_tcrdist_no_pca = ((avg_tcrdist - avg_no_pca) / avg_no_pca) * 100

# Plotting
data = [auc_no_pca, auc_pca, auc_umap, auc_sete,auc_tcrdist]
fig, ax = plt.subplots()
ax.boxplot(data, labels=['No PCA', 'PCA', 'UMAP', 'SETE','TCRdist3'])
ax.set_ylabel('AUC Scores')
ax.set_title('Comparison of AUC Scores')

# Show average values on the plot
#for i, avg in enumerate([avg_no_pca, avg_pca, avg_umap], start=1):
    #ax.text(i, avg + 0.01, f'{avg:.3f}', ha='center', va='bottom', fontsize=9)

# Show the plot
plt.show()

(avg_no_pca, avg_pca, avg_umap,avg_sete,avg_tcrdist), (perc_diff_pca_no_pca, perc_diff_umap_no_pca,perc_diff_sete_no_pca,perc_diff_tcrdist_no_pca)

# also need to finish extension stuff and do epitope analysis stuff, can be found on final_outputs.py